## Self Supervised Pretraining for Tabular Data

We have implemented two Self Supervised Pre-training routines that allow the user to pre-train *all* tabular models in the library with the exception of the TabPerceiver (which is a special monster).

The two routines implemented are illustrated in the figures below. The 1st is from [TabNet: Attentive Interpretable Tabular Learnin](https://arxiv.org/abs/1908.07442) and is designed for models that do not use transformer-based architectures, while the second is from [SAINT: Improved Neural Networks for Tabular Data via Row Attention and Contrastive Pre-Training](https://arxiv.org/abs/2106.01342), and is designed for models that use transformer-based architectures.

<p align="center">
  <img width="800" src="figures/self_supervised_tabnet.png">
</p>

Fig 1. Figure 2 in their paper. I have included de original caption in case is useful, althought the Figure itself is pretty self explanatory


<p align="center">
  <img width="600" src="figures/self_supervised_saint.png">
</p>

Fig 2. Figure 1 in their paper. Here the caption is necessary 😏


It is beyond the scope of this notebook to explain in detail those implementations. Therefore, we strongly recommend the user to go and read the papers if this functionality is of interest to her/him.

One thing is worth noticing however. As seen in Fig 1(the TabNet paper's Fig 2) the masking of the input features happens in the feature space. However, the implementation in this library is inspired by that at the [dreamquark-ai](https://github.com/dreamquark-ai/tabnet) repo, which is in itself inspired by the original implementation (by the way, at this point I will write it once again. All TabNet related things in this library are inspired when not directly based in the code in that repo, therefore, ALL CREDIT TO THE GUYS AT dreamquark-ai).

In that implementation the masking happens in the embedding space, and currently does not mask the entire embedding (i.e. categorical feature). We decided to release as it is in this version and we will implement the exact same process described in the paper in future releases. 

Having said all of the above let's see how to use self supervision for tabular data with `pytorch-widedeep`. We will concentrate in this notebook on the 2nd of the two approaches (the 'SAINT approach'). For details on the 1st approach (the 'TabNet' approach) please see `16_Self_Supervised_Pretraning_pt1`.

### Self Supervision transformer-based models..

...or in general, for models where the embeddigns have all the same dimensions. In this library, these are:

- TabTransformer
- FTTransformer
- SAINT
- TabFastFormer

Note that there is one additional Transformer-based model, the `TabPerceiver`, however this is a "particular" model and at the moment we do not support self supervision for it, but it will come. 

Let see at one example using the `FTTransformer`. 

In [1]:
import torch
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from pytorch_widedeep import Trainer
from pytorch_widedeep.models import WideDeep, FTTransformer
from pytorch_widedeep.metrics import Accuracy
from pytorch_widedeep.datasets import load_adult
from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.self_supervised_training import (
    ContrastiveDenoisingTrainer,
)

/Users/javierrodriguezzaurin/.pyenv/versions/3.8.12/envs/wd38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = load_adult(as_frame=True)
df.columns = [c.replace("-", "_") for c in df.columns]
df["income_label"] = (df["income"].apply(lambda x: ">50K" in x)).astype(int)
df.drop("income", axis=1, inplace=True)

# one could chose to use a validation set for early stopping, hyperparam
# optimization, etc. This is just an example, so we simply use train/test
# split
df_tr, df_te = train_test_split(df, test_size=0.2, stratify=df.income_label)

cat_embed_cols = [
    "workclass",
    "education",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "gender",
    "capital_gain",
    "capital_loss",
    "native_country",
]
continuous_cols = ["age", "hours_per_week"]
target_col = "income_label"

tab_preprocessor = TabPreprocessor(
    cat_embed_cols=cat_embed_cols,
    continuous_cols=continuous_cols,
    with_attention=True,
    with_cls_token=True,  # this is optional
)
X_tab = tab_preprocessor.fit_transform(df_tr)
target = df_tr[target_col].values

In [3]:
ft_transformer = FTTransformer(
    column_idx=tab_preprocessor.column_idx,
    cat_embed_input=tab_preprocessor.cat_embed_input,
    continuous_cols=tab_preprocessor.continuous_cols,
    input_dim=32,
    kv_compression_factor=0.5,
    n_blocks=3,
    n_heads=4,
)

In [4]:
# for a full list of the params for the the ContrastiveDenoisingTrainer (which are many) please see the docs.
# Note that using these params involves some knowledge of the routine and the architecture of the model used
contrastive_denoising_trainer = ContrastiveDenoisingTrainer(
    model=ft_transformer,
    preprocessor=tab_preprocessor,
)
contrastive_denoising_trainer.pretrain(X_tab, n_epochs=5, batch_size=256)

epoch 5: 100%|████████████████████████████████████████████████████████████████████████████████████| 153/153 [00:21<00:00,  6.98it/s, loss=7.89]


In [5]:
contrastive_denoising_trainer.save(
    path="pretrained_weights", model_filename="contrastive_denoising_model.pt"
)

some time has passed

In [6]:
# some time has passed, we load the model with torch as usual:
contrastive_denoising_model = torch.load(
    "pretrained_weights/contrastive_denoising_model.pt"
)

NOW, AND THIS IS IMPORTANT! We have loaded the entire contrastive, denoising model. To proceed to the supervised training we ONLY need the attention-based model, which is the 'model' attribute of the trainer, let's have a look

In [7]:
contrastive_denoising_model.model

FTTransformer(
  (cat_and_cont_embed): SameSizeCatAndContEmbeddings(
    (cat_embed): SameSizeCatEmbeddings(
      (embed): Embedding(323, 32, padding_idx=0)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (cont_norm): Identity()
    (cont_embed): ContEmbeddings(2, 32, embed_dropout=0.1, use_bias=True)
  )
  (encoder): Sequential(
    (fttransformer_block0): FTTransformerEncoder(
      (attn): LinearAttention(
        (dropout): Dropout(p=0.2, inplace=False)
        (qkv_proj): Linear(in_features=32, out_features=96, bias=False)
        (out_proj): Linear(in_features=32, out_features=32, bias=False)
      )
      (ff): FeedForward(
        (w_1): Linear(in_features=32, out_features=84, bias=True)
        (w_2): Linear(in_features=42, out_features=32, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (activation): REGLU()
      )
      (attn_normadd): NormAdd(
        (dropout): Dropout(p=0.2, inplace=False)
        (ln): LayerNorm((32,), eps=1e-05, elementwis

In [8]:
pretrained_model = contrastive_denoising_model.model

In [9]:
# and as always, ANY supervised model in this library has to go throuth the WideDeep class:
model = WideDeep(deeptabular=pretrained_model)
trainer = Trainer(model=model, objective="binary", metrics=[Accuracy])

trainer.fit(X_tab=X_tab, target=target, n_epochs=5, batch_size=256)

# And, you know...we get a test metric
X_tab_te = tab_preprocessor.transform(df_te)
target_te = df_te[target_col].values

preds = trainer.predict(X_tab=X_tab_te)
test_acc = accuracy_score(target_te, preds)
print(test_acc)

predict: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 58.57it/s]


0.8705087521752483
